In [1]:
import pandas as pd
import numpy as np
from pulp import *
import os
import pulp as lp
# import cplex
# # from cplex import Cplex
# # from cplex.exceptions import CplexError


C:\Users\Anu Zan\AppData\Local\Temp\ipykernel_2428\592519910.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import sys
sys.path.append(r"C:\\Program Files\\IBM\\ILOG\\CPLEX_Studio2211\\python\setup.py install")  # Update the path with your actual path



In [3]:
##Load datasets

# Change the working directory to the notebook's directory
notebook_dir = "C:\\Users\\Anu Zan\\Desktop\\GeorgiaTech Spring 2024\\ISYE 6740 CDA\\project"
os.chdir(notebook_dir)

# Now, define the relative path to the dataset
dataset_path = os.path.join("..", "Dataset")

Optimization_dataset=pd.read_csv(os.path.join(dataset_path, "OptimizationData_04012024.csv"))
Intervention_dataset=pd.read_csv(os.path.join(dataset_path,"InterventionData_04052024.csv"))

#removed last column(P_NOTX) since it does not appear in the intervention dataset
#Household_dataset=Household_dataset.iloc[:, :-1]

In [4]:
## Data cleaning and conversions to datetype
Optimization_dataset['intervention_eligibility_start'] = pd.to_datetime(Optimization_dataset['intervention_eligibility_start'])
Optimization_dataset['intervention_eligibility_end'] = pd.to_datetime(Optimization_dataset['intervention_eligibility_end'])
Intervention_dataset['EnrollmentDate'] = pd.to_datetime(Intervention_dataset['EnrollmentDate'])
Optimization_dataset

,Unnamed: 0,ID,intervention_eligibility_start,intervention_eligibility_end,subpopulation,P_RRH,P_TSH,P_PSH,P_NOTX
0,0,5346,2020-08-12,2020-11-12,0,0.403523,0.365126,0.317578,0.038237
1,1,10810,2020-06-14,2020-09-14,1,0.486524,0.312525,0.308390,0.086466
2,2,13875,2021-07-25,2022-11-08,0,0.390293,0.380058,0.247997,0.040080
3,3,13440,2020-06-20,2021-08-21,0,0.343476,0.388433,0.295636,0.026810
4,4,11434,2020-06-17,2020-09-17,1,0.340353,0.415589,0.271931,0.026012
...,...,...,...,...,...,...,...,...,...
20676,20890,257,2020-06-10,2022-03-09,1,0.614422,0.373384,0.398029,0.238472
20677,20891,257,2022-11-09,2023-01-03,1,0.563452,0.379900,0.383452,0.252566
20678,20892,12780,2020-06-21,2020-09-21,1,0.483045,0.375423,0.282675,0.034415
20679,20893,2256,2020-06-09,2020-09-09,1,0.495911,0.397076,0.272658,0.052616


In [5]:
## Remove rows where intervention ends in 2023
Training_Optimization_dataset=Optimization_dataset[Optimization_dataset['intervention_eligibility_end'] < "2021-12-31"]
Training_intervention_dataset=Intervention_dataset[Intervention_dataset['EnrollmentDate']< "2021-12-31" ]


In [6]:
## Groupe intervention type per week 
Training_intervention_dataset.sort_values('EnrollmentDate', inplace=True)
Training_intervention_dataset_grouped=Training_intervention_dataset.groupby([pd.Grouper(key='EnrollmentDate', freq='W'), 'ProjectTypeName']).size().unstack(fill_value=0)
# Create a new row as a DataFrame with the correct datetime index directly
new_row_index = pd.DatetimeIndex([pd.to_datetime('2017-01-01')])
new_row = pd.DataFrame({'PH - Permanent Supportive Housing': [0],
                        'PH - Rapid Re-Housing': [0],
                        'Transitional Housing': [0]},
                       index=new_row_index)

# Concatenate the new_row DataFrame with the existing grouped DataFrame
# Since your grouped DataFrame has a datetime index (EnrollmentDate), ensure the new row also does
Training_intervention_dataset_grouped = pd.concat([new_row, Training_intervention_dataset_grouped])

# Since concatenation can mess up the chronological order, sort by index just in case
Training_intervention_dataset_grouped.sort_index(inplace=True)

# Check to ensure the new row is added correctly
Training_intervention_dataset_grouped.head().dropna()
int

C:\Users\Anu Zan\AppData\Local\Temp\ipykernel_2428\2728604530.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Training_intervention_dataset.sort_values('EnrollmentDate', inplace=True)


int

In [7]:
Training_Optimization_dataset.sort_values('intervention_eligibility_start', inplace=True)

# Now, grouping by week and applying the provided lambda function to reset index within each group, but ensuring 'Week_Start_Date' is preserved
Training_Optimization_dataset_grouped = Training_Optimization_dataset.set_index("intervention_eligibility_start").groupby(pd.Grouper(freq='W')).apply(lambda _df: _df.assign(Week_Start=_df.name)).reset_index(drop=True)

# The 'Week_Start' column now contains the start of the week for the 'intervention_eligibility_start' date
Training_Optimization_dataset_grouped.dropna()

C:\Users\Anu Zan\AppData\Local\Temp\ipykernel_2428\261131170.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Training_Optimization_dataset.sort_values('intervention_eligibility_start', inplace=True)


,Unnamed: 0,ID,intervention_eligibility_end,subpopulation,P_RRH,P_TSH,P_PSH,P_NOTX,Week_Start
0,7681,13861,2017-03-31,1,0.605708,0.392315,0.390282,0.081395,2017-01-01
1,18382,12417,2017-04-01,1,0.384913,0.323653,0.331396,0.078390,2017-01-01
2,8521,5981,2018-10-03,1,0.387764,0.385844,0.360858,0.066253,2017-01-08
3,19079,9994,2017-06-28,1,0.364034,0.396487,0.285617,0.032998,2017-01-08
4,18327,4867,2017-10-23,1,0.459237,0.369800,0.362774,0.101124,2017-01-08
...,...,...,...,...,...,...,...,...,...
13403,18790,10317,2020-10-05,0,0.486860,0.435341,0.424961,0.373232,2023-12-03
13404,10530,11974,2017-06-01,1,0.423834,0.379707,0.309554,0.148024,2023-12-10
13405,14315,2622,2021-08-01,0,0.417905,0.394632,0.310507,0.113796,2023-12-17
13406,10406,15750,2017-09-08,1,0.474584,0.403793,0.272180,0.126019,2023-12-24


In [8]:
## Mapping 2 datasets on intervention type
intervention_mapping = {
    'PH - Rapid Re-Housing': 'P_RRH',
    'Transitional Housing': 'P_TSH',
    'PH - Permanent Supportive Housing': 'P_PSH',
    #  P_NOTX 
}

In [9]:
intervention_types = ['PH - Permanent Supportive Housing', 'PH - Rapid Re-Housing', 'Transitional Housing']

In [10]:
# Initialize the problem
prob = lp.LpProblem("Housing_Intervention_Optimization", lp.LpMaximize)

In [11]:
household_ids = Training_Optimization_dataset_grouped['ID'].unique()
intervention_types = ['PH - Permanent Supportive Housing', 'PH - Rapid Re-Housing', 'Transitional Housing']  # Assuming these are your interventions
weeks = Training_Optimization_dataset_grouped['Week_Start'].dt.strftime('%Y-%m-%d').unique()


In [12]:
weeks

array(['2017-01-01', '2017-01-08', '2017-01-15', '2017-01-22',
       '2017-01-29', '2017-02-05', '2017-02-12', '2017-02-19',
       '2017-02-26', '2017-03-05', '2017-03-12', '2017-03-19',
       '2017-03-26', '2017-04-02', '2017-04-09', '2017-04-16',
       '2017-04-23', '2017-04-30', '2017-05-07', '2017-05-14',
       '2017-05-21', '2017-05-28', '2017-06-04', '2017-06-11',
       '2017-06-18', '2017-06-25', '2017-07-02', '2017-07-09',
       '2017-07-16', '2017-07-23', '2017-07-30', '2017-08-06',
       '2017-08-13', '2017-08-20', '2017-08-27', '2017-09-03',
       '2017-09-10', '2017-09-17', '2017-09-24', '2017-10-01',
       '2017-10-08', '2017-10-15', '2017-10-22', '2017-10-29',
       '2017-11-05', '2017-11-12', '2017-11-19', '2017-11-26',
       '2017-12-03', '2017-12-10', '2017-12-17', '2017-12-24',
       '2017-12-31', '2018-01-07', '2018-01-14', '2018-01-21',
       '2018-01-28', '2018-02-04', '2018-02-11', '2018-02-18',
       '2018-02-25', '2018-03-04', '2018-03-11', '2018-

In [13]:
# Define decision variables
x = LpVariable.dicts("Assignment", 
                     [(i, j, t) for i in household_ids 
                                  for j in intervention_types 
                                  for t in weeks], cat='Binary')


In [14]:
probabilities = {}
for index, row in Training_Optimization_dataset_grouped.iterrows():
    week_str = row['Week_Start'].strftime('%Y-%m-%d')
    
    # Check if the week exists in the Training_intervention_dataset_grouped DataFrame
    if week_str in Training_intervention_dataset_grouped.index:
        # Access the row for the specific week, which gives a Series with intervention capacities
        available_interventions = Training_intervention_dataset_grouped.loc[week_str]
        
        # Iterate through the intervention mapping to assign probabilities
        for intervention, p_intervention in intervention_mapping.items():
            # Check directly if the intervention has a non-zero capacity for the week
            # The 'get' method on Series is used to safely attempt to access an intervention, returning None if not found
            if available_interventions.get(intervention, 0) > 0:
                # Check if the probability value exists and is non-negative
                if pd.notnull(row[p_intervention]):
                    # Assign the probability to the dictionary
                    probabilities[(row['ID'], intervention, week_str)] = row[p_intervention]


In [15]:
probabilities 

{(5981, 'PH - Rapid Re-Housing', '2017-01-08'): 0.3877637417636208,
 (5981, 'PH - Permanent Supportive Housing', '2017-01-08'): 0.360857620938829,
 (9994, 'PH - Rapid Re-Housing', '2017-01-08'): 0.364033611625594,
 (9994, 'PH - Permanent Supportive Housing', '2017-01-08'): 0.2856171371616515,
 (4867, 'PH - Rapid Re-Housing', '2017-01-08'): 0.4592371047897797,
 (4867, 'PH - Permanent Supportive Housing', '2017-01-08'): 0.3627736540996712,
 (11516, 'PH - Rapid Re-Housing', '2017-01-08'): 0.5132157230996602,
 (11516,
  'PH - Permanent Supportive Housing',
  '2017-01-08'): 0.2697711391526141,
 (15823, 'PH - Rapid Re-Housing', '2017-01-08'): 0.2891406306892777,
 (15823,
  'PH - Permanent Supportive Housing',
  '2017-01-08'): 0.2952944096649059,
 (618, 'PH - Rapid Re-Housing', '2017-01-08'): 0.5119365997053356,
 (618, 'PH - Permanent Supportive Housing', '2017-01-08'): 0.3204713463818098,
 (1593, 'PH - Rapid Re-Housing', '2017-01-08'): 0.4788642465276124,
 (1593, 'PH - Permanent Supportive H

In [16]:
# Define the objective function
prob += lp.lpSum([probabilities[(i, j, t)] * x[(i, j, t)]
                  for i in household_ids 
                  for j in intervention_types 
                  for t in weeks 
                  if (i, j, t) in probabilities]), "Total_Probability_of_Exiting_Homelessness"


In [17]:
# Building the capacities dictionary
capacities = {}
for t, row in Training_intervention_dataset_grouped.iterrows():
    week_str = t.strftime('%Y-%m-%d')
    for j in intervention_types:  
        capacities[(j, week_str)] = row[j]

In [18]:
# Capacity Constraints
for j in intervention_types:  
    for week_str in weeks:
        if (j, week_str) in capacities:  # Ensure the capacity data exists for this intervention and week
            prob += lp.lpSum(x[(i, j, week_str)] for i in household_ids) <= capacities[(j, week_str)], f"Capacity_{j}_{week_str}"



In [19]:
# Additional binary decision variables for each household and week
assigned_up_to_week = LpVariable.dicts("AssignedUpToWeek", 
                                       [(i, t) for i in household_ids for t in weeks], 
                                       cat='Binary')

# Single Assignment for Each Household
for i in household_ids:
    prob += lp.lpSum(assigned_up_to_week[(i, t)] for t in weeks) <= 1, f"Single_Assignment_{i}"



In [20]:
for i in household_ids:
    for week_idx, t in enumerate(weeks):
        # If assigned this week or any previous week, then no further assignments
        prob += assigned_up_to_week[(i, t)] >= lp.lpSum(x[(i, j, t)] for j in intervention_types), f"Assign_This_Week_{i}_{t}"

In [21]:
prob.solve(pulp.PULP_CBC_CMD(presolve=1, msg=True))

1

In [22]:
# Check the status of the solution
print("Solution Status:", LpStatus[prob.status])

# Print the objective function value (total probability of exiting homelessness)
print("Total Probability of Exiting Homelessness:", value(prob.objective))

# Iterate through the decision variables and safely print non-zero assignments
for variable in prob.variables():
    if variable.varValue is not None and variable.varValue > 0:
        print(variable.name, "=", variable.varValue)


Solution Status: Optimal
Total Probability of Exiting Homelessness: 464.526690769765
AssignedUpToWeek_(0,_'2021_02_28') = 1.0
AssignedUpToWeek_(10000,_'2017_08_13') = 1.0
AssignedUpToWeek_(10001,_'2019_03_17') = 1.0
AssignedUpToWeek_(10002,_'2019_01_27') = 1.0
AssignedUpToWeek_(10003,_'2019_03_17') = 1.0
AssignedUpToWeek_(10004,_'2021_08_08') = 1.0
AssignedUpToWeek_(10005,_'2020_10_18') = 1.0
AssignedUpToWeek_(10007,_'2020_08_16') = 1.0
AssignedUpToWeek_(10008,_'2020_01_19') = 1.0
AssignedUpToWeek_(10009,_'2017_03_26') = 1.0
AssignedUpToWeek_(10010,_'2021_02_07') = 1.0
AssignedUpToWeek_(10014,_'2020_04_19') = 1.0
AssignedUpToWeek_(10015,_'2017_02_26') = 1.0
AssignedUpToWeek_(10017,_'2020_08_02') = 1.0
AssignedUpToWeek_(1002,_'2020_09_06') = 1.0
AssignedUpToWeek_(10021,_'2017_07_16') = 1.0
AssignedUpToWeek_(10023,_'2020_04_26') = 1.0
AssignedUpToWeek_(10024,_'2019_03_17') = 1.0
AssignedUpToWeek_(10025,_'2020_02_16') = 1.0
AssignedUpToWeek_(10027,_'2018_01_21') = 1.0
AssignedUpToWeek_(10

In [23]:
# Quick check to see if the probabilities dictionary is populated
print("Sample probabilities:", list(probabilities.items())[:5])


Sample probabilities: [((5981, 'PH - Rapid Re-Housing', '2017-01-08'), 0.3877637417636208), ((5981, 'PH - Permanent Supportive Housing', '2017-01-08'), 0.360857620938829), ((9994, 'PH - Rapid Re-Housing', '2017-01-08'), 0.364033611625594), ((9994, 'PH - Permanent Supportive Housing', '2017-01-08'), 0.2856171371616515), ((4867, 'PH - Rapid Re-Housing', '2017-01-08'), 0.4592371047897797)]


In [24]:
# Ensure that the keys (household_id, intervention_type, week) used in defining 'x' align with 'probabilities'
print("Sample decision variable keys:", [key for key in x.keys()][:5])


Sample decision variable keys: [(13861, 'PH - Permanent Supportive Housing', '2017-01-01'), (13861, 'PH - Permanent Supportive Housing', '2017-01-08'), (13861, 'PH - Permanent Supportive Housing', '2017-01-15'), (13861, 'PH - Permanent Supportive Housing', '2017-01-22'), (13861, 'PH - Permanent Supportive Housing', '2017-01-29')]


In [25]:
# Check for any non-numeric values that might be causing issues
non_numeric = {k: v for k, v in probabilities.items() if not isinstance(v, (int, float))}
print("Non-numeric values in probabilities (if any):", non_numeric)


Non-numeric values in probabilities (if any): {}


In [26]:
# Initialize an empty list to store assignment summaries
assignment_summaries = []

# Iterate through the decision variables and extract non-zero assignments
for variable in prob.variables():
    if variable.varValue is not None and variable.varValue > 0:
        assignment_summaries.append((variable.name, variable.varValue))

# Print the assignment summaries
print("Assignment Summaries:")
for summary in assignment_summaries:
    print(summary)

Assignment Summaries:
("AssignedUpToWeek_(0,_'2021_02_28')", 1.0)
("AssignedUpToWeek_(10000,_'2017_08_13')", 1.0)
("AssignedUpToWeek_(10001,_'2019_03_17')", 1.0)
("AssignedUpToWeek_(10002,_'2019_01_27')", 1.0)
("AssignedUpToWeek_(10003,_'2019_03_17')", 1.0)
("AssignedUpToWeek_(10004,_'2021_08_08')", 1.0)
("AssignedUpToWeek_(10005,_'2020_10_18')", 1.0)
("AssignedUpToWeek_(10007,_'2020_08_16')", 1.0)
("AssignedUpToWeek_(10008,_'2020_01_19')", 1.0)
("AssignedUpToWeek_(10009,_'2017_03_26')", 1.0)
("AssignedUpToWeek_(10010,_'2021_02_07')", 1.0)
("AssignedUpToWeek_(10014,_'2020_04_19')", 1.0)
("AssignedUpToWeek_(10015,_'2017_02_26')", 1.0)
("AssignedUpToWeek_(10017,_'2020_08_02')", 1.0)
("AssignedUpToWeek_(1002,_'2020_09_06')", 1.0)
("AssignedUpToWeek_(10021,_'2017_07_16')", 1.0)
("AssignedUpToWeek_(10023,_'2020_04_26')", 1.0)
("AssignedUpToWeek_(10024,_'2019_03_17')", 1.0)
("AssignedUpToWeek_(10025,_'2020_02_16')", 1.0)
("AssignedUpToWeek_(10027,_'2018_01_21')", 1.0)
("AssignedUpToWeek_(100